In [2]:
import sys
import os
sys.path.append(os.path.abspath('../'))
from sklearn.preprocessing import StandardScaler
import functions_ml as fml
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
# 1. Get dataset
results = pd.read_csv('../datasets/instances_results.csv')
features = pd.read_csv('../datasets/multi_plant_instance_features.csv')
dataset = fml.create_dataset(features, results)
TOLERANCE_LIMIT = 0.01 / 100
dataset = fml.create_multi_label_target(dataset, TOLERANCE_LIMIT)


In [4]:
# Select instance-based features (excluding labels or unrelated columns)
feature_columns = [col for col in dataset.columns if not col.startswith('RF_') and col != 'instance']
X = dataset[feature_columns]
X = fml.feature_selection(X)

In [6]:
# Compute the correlation matrix
corr_matrix = X.corr().abs()  # Use absolute values to consider positive/negative correlations equally

# Set a threshold for "high correlation"
threshold = 0.8

# Find feature pairs with correlation above the threshold
high_corr_pairs = []
for i in range(corr_matrix.shape[0]):
    for j in range(i + 1, corr_matrix.shape[1]):  # Only look at the upper triangle
        if corr_matrix.iloc[i, j] > threshold:
            feature_1 = corr_matrix.index[i]
            feature_2 = corr_matrix.columns[j]
            correlation_value = corr_matrix.iloc[i, j]
            high_corr_pairs.append((feature_1, feature_2, correlation_value))

# Convert the high-correlation pairs into a DataFrame for easy visualization
high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

# Sort by Feature 1 alphabetically, and then by Correlation (descending)
high_corr_df = high_corr_df.sort_values(by=['Feature 1', 'Correlation'], ascending=[True, False])

# Display the high-correlation table
print("Highly Correlated Features (Sorted by Feature 1):")
print(high_corr_df)

Highly Correlated Features (Sorted by Feature 1):
                 Feature 1                Feature 2  Correlation
1               avg_demand               p50_demand     0.836055
7          avg_utilization          p75_utilization     0.839885
6          avg_utilization          p25_utilization     0.815972
2              cv_capacity          std_utilization     0.811481
8          max_utilization          std_utilization     0.871760
3           min_setup_cost           avg_setup_cost     0.869983
5          min_utilization          std_utilization     0.857777
0               num_plants  std_transportation_cost     0.875355
4  std_transportation_cost  iqr_transportation_cost     0.812355
